### Tokenize Data and Prepare it for FastAlign

In [4]:
%run ../scripts/preprocessing.py

print(spanish_pos[0:3])

¡Venidos buenos!
Kim bive kon Ken.
Me yamo Ichiro Tanaka.
Eyos me yaman Bob.
Me yamo Wang.
Si, me yamo Karen Smith.
Ninguno no vino.
Avlo japanez, inglez i fransez.
No, no me yamo María. Me yamo Teresa.
Eyos me yaman Bob.
Eya es muy yakishikliya.
Kero avlar kon eya.
Munchos kantes relijiozos en Ladino son traduksiones del Ivrit.
No me akodro lo ke komi anoche.
La mujer solo save kuzinar el arroz.
Kero bivir en la sivdad.
Me azes reir muncho.
Sos muy mazalózo.
Eya es una mujer dobra.
En este momento estamos lavorando.
Tyenes muy buena suerte.
Sos muy mazalóza.
Te gusta el beisbol?
Bivo en Turkiya.
Kero bivir en Ostralya.
Ninguno no vino.
Él es uertolano.
Me gusta la sus kaza.
Kamma?
Eskucha a tu madre!
A mi mama le gustava muncho kontar anekdotas.
El konjelador esta en el garaj.
El búto de kada vida es la muerte.
Te puedes fiar en eyos.
La lingua ofisiala de Israel es el ivrit moderno.
Kero skrivir un artikulo.
La sivdad kapitala de Fransya es Paris.
Aki Yerushalayim es una revista kult